In [49]:
from selenium import webdriver


driver = webdriver.Chrome()
driver.get("https://www.naver.com")



In [42]:
# driver.back()


In [43]:
# driver.forward()


In [36]:
driver.current_url

'https://www.google.com/'

In [61]:
driver.title






'NAVER'

In [70]:
#query
from selenium.webdriver.common.by import By

# CSS 셀렉터로 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, "#query")




In [71]:
# 검색창 클릭
search.click()


# 검색어 입력
search.send_keys("연세대학교")

from selenium.webdriver.common.keys import Keys  # Keys 클래스 import 추가
# 검색 실행을 위해 엔터키 입력 
search.send_keys(Keys.ENTER)

# 검색 결과 상위 5개 항목 찾기
results = driver.find_elements(By.CSS_SELECTOR, "#main .item")[:5]

# 검색 결과 출력
for result in results:
    print(result.text)



In [78]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys  # Keys 클래스 import 추가


driver = webdriver.Chrome()
driver.get("https://www.naver.com")

# driver.refresh()


# CSS 셀렉터로 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, "#query")

# 검색창 클릭
search.click()


# 검색어 입력
search.send_keys("연세대학교")

from selenium.webdriver.common.keys import Keys  # Keys 클래스 import 추가
# 검색 실행을 위해 엔터키 입력 
search.send_keys(Keys.ENTER)



In [75]:



# 해외축구(EPL) URL: 
# https://m.sports.naver.com/wfootball/schedule/index?category=epl&date=2024-11-23
# EPL cheer 예제
# https://m.sports.naver.com/game/2024111110050850171/cheer

#NBA URL
#https://m.sports.naver.com/basketball/schedule/index?category=nba&date=2024-11-22

#NBA URL cheer 예제
# https://m.sports.naver.com/game/2024111202/cheer

In [77]:
import requests

def check_robots_txt(url):
    try:
        response = requests.get(f"{url}/robots.txt")
        print(response.text)
    except Exception as e:
        print(f"에러 발생: {e}")

# 네이버 스포츠 robots.txt 확인
check_robots_txt("https://m.sports.naver.com")

User-agent: *
Disallow: /
Allow: /$
Allow: /index



In [ ]:

def crawl_schedule(category='epl'):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    
    try:
        today = datetime.now().strftime("%Y-%m-%d")
        url = f"https://m.sports.naver.com/wfootball/schedule/index?category={category}&date={today}"
        driver.get(url)
        time.sleep(2)
        
        matches = []
        game_elements = driver.find_elements(By.CSS_SELECTOR, ".game_item")
        
        for game in game_elements:
            try:
                game_id = game.get_attribute("data-game-id")
                home_team = game.find_element(By.CSS_SELECTOR, ".home_team").text
                away_team = game.find_element(By.CSS_SELECTOR, ".away_team").text
                match_time = game.find_element(By.CSS_SELECTOR, ".time").text
                
                match_info = {
                    "game_id": game_id,
                    "league": "EPL" if category == "epl" else "NBA",
                    "home_team": home_team,
                    "away_team": away_team,
                    "match_time": match_time,
                    "cheer_url": f"https://m.sports.naver.com/game/{game_id}/cheer"
                }
                matches.append(match_info)
                
            except Exception as e:
                print(f"매치 파싱 오류: {e}")
                
        return matches
    finally:
        driver.quit()


#### 해외축구(EPL) URL: 
##### https://m.sports.naver.com/wfootball/schedule/index?category=epl&date=2024-11-23

#### NBA URL:
##### https://m.sports.naver.com/basketball/schedule/index?category=nba&date=2024-11-22

#### EPL cheer 예제:
##### https://m.sports.naver.com/game/2024111110050850171/cheer

#### NBA cheer 예제:
##### https://m.sports.naver.com/game/2024111202/cheer

In [80]:
## 여기서부터 진짜 크롤링

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys  # Keys 클래스 import 추가
from selenium.webdriver.chrome.options import Options
import time
import json
from datetime import datetime



In [129]:

driver = webdriver.Chrome()

today = datetime.now()
category = "epl"

current_month = today.month
current_year = today.year

for i in range(6):  # 현재 달부터 5개월 후까지
    target_month = (current_month + i) % 12 or 12  # 12월 다음은 1월
    target_year = current_year + (current_month + i - 1) // 12  # 연도 변경 처리
    
    date = datetime(target_year, target_month, 1).strftime("%Y-%m-%d")
    url = f"https://m.sports.naver.com/wfootball/schedule/index?category={category}&date={date}"
    driver.get(url)
    time.sleep(2)
    elements = driver.find_elements(By.CSS_SELECTOR, ".ScheduleLeagueType_title__2Kalm")
    match_elements = driver.find_elements(By.CSS_SELECTOR, ".ScheduleLeagueType_match_list__1-n6x")
    for date_element, match_element in zip(elements, match_elements):
        print(f"\n[{date_element.text}] 경기일정:")  # 날짜 출력
        # li 요소들 찾기
        li_elements = match_element.find_elements(By.CSS_SELECTOR, "li")
        for idx, li in enumerate(li_elements):
            match_time = li.find_element(By.CSS_SELECTOR, ".MatchBox_time__nIEfd")
            match_status = li.find_element(By.CSS_SELECTOR, ".MatchBox_status__2pbzi")
            match_area = li.find_element(By.CSS_SELECTOR, ".MatchBox_match_area__39dEr")
            match_areas = match_area.find_elements(By.CSS_SELECTOR, ".MatchBoxTeamArea_team__3aB4O")
            link = match_area.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            actual_time = match_time.get_attribute("textContent").strip().split("\n")[-1][-5:]

            print(f"경기날짜: {date_element.text}")
            print(f"경기 시간: {actual_time}")
            print(f"경기 상태: {match_status.text}")
            print(f"매치 정보: 홈 {match_areas[0].text} vs 원정 {match_areas[1].text}")
            print(f"응원 링크: {link}/cheer")
            print("-" * 50)
    time.sleep(1)

#content > div > div:nth-child(5) > div > div:nth-child(1) > ul > li:nth-child(1) > div > div.MatchBox_match_area__39dEr






[11월 2일 (토)] 경기일정:
경기날짜: 11월 2일 (토)
경기 시간: 21:30
경기 상태: 종료
매치 정보: 홈 뉴캐슬 vs 원정 아스널
응원 링크: https://m.sports.naver.com/game/2024110210050850157/cheer
--------------------------------------------------

[11월 3일 (일)] 경기일정:
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 본머스 vs 원정 맨시티
응원 링크: https://m.sports.naver.com/game/2024110310050850147/cheer
--------------------------------------------------
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 입스위치 vs 원정 레스터 시티
응원 링크: https://m.sports.naver.com/game/2024110310050850151/cheer
--------------------------------------------------
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 리버풀 vs 원정 브라이턴
응원 링크: https://m.sports.naver.com/game/2024110310050850153/cheer
--------------------------------------------------
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 노팅엄 vs 원정 웨스트햄
응원 링크: https://m.sports.naver.com/game/2024110310050850159/cheer
--------------------------------------------------
경기날짜: 11월 3일 (일)
경기 시간: 00:00
경기 상태: 종료
매치 정보: 홈 사우샘프턴 vs 원